<a href="https://colab.research.google.com/github/Elman295/Vision-Mamba-MNIST-/blob/main/vim_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mamba_ssm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn
from mamba_ssm import Mamba

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
x = torch.rand(size = (16,100,200), device =device)

mabda = Mamba(200).to(device)
y = mabda(x)

In [ ]:
y.shape

torch.Size([16, 100, 200])

In [ ]:
train_d = datasets.MNIST(
    root = "data",
    train = True,
    download = True,
    transform = transforms.ToTensor()
)

test_d = datasets.MNIST(
    root = "data",
    train = False,
    download = True,
    transform = transforms.ToTensor()
)

100%|██████████| 9912422/9912422 [00:00<00:00, 183389622.92it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 32038004.19it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 41920082.66it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 17606773.35it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
len(train_d)

60000

In [ ]:
len(test_d)

10000

In [ ]:
image, label = train_d[0]

In [ ]:
image.shape

torch.Size([1, 28, 28])

In [ ]:
x = torch.rand(size = (1,28,28))


def patchify(data, p_size):

  c,h,w = data.shape
  ph,pw = p_size, p_size

  np = (h // ph) * (w // pw)

  data = data.view((c,np,ph,pw))
  return data


y = patchify(x,4)

In [ ]:
y.shape

torch.Size([1, 49, 4, 4])

In [ ]:
class DS(Dataset):

  def __init__(self, data, transform = None):
    self.data = data
    self.transform = transform

  def __len__(self):
    return len(self.data)


  def patchify(self, data, p_size = 4):


    c,h,w = data.shape
    ph,pw = p_size, p_size

    np = (h // ph) * (w // pw)

    data = data.view((c,np,ph,pw))
    return data


  def __getitem__(self, idx):
    image, label = self.data[idx]

    if self.transform is not None:
      image = self.transform(image)

    image = self.patchify(image)
    image = image.view((49,16))

    return image, label

In [ ]:
train_ds = DS(train_d)

test_ds = DS(test_d)

In [ ]:
i,l = train_ds[0]

In [ ]:
i.shape

torch.Size([49, 16])

In [ ]:
train_dl = DataLoader(dataset = train_ds, batch_size = 128, shuffle = True)

test_dl =DataLoader(dataset = test_ds, batch_size = 128, shuffle = False)

In [ ]:
image, label = next(iter(train_dl))

In [ ]:
print(image.shape)

torch.Size([128, 49, 16])


In [ ]:
x = torch.rand(size = (128,49,16))
linear = nn.Linear(16,128)
y = linear(x)

In [ ]:
y.shape

torch.Size([128, 49, 128])

In [ ]:
class ViT(nn.Module):

  def __init__(self):
    super(ViT, self).__init__()
    self.linear_1 = nn.Linear(16,128)
    # self.encoder_layer = nn.TransformerEncoderLayer(128,4)
    # self.encoder = nn.TransformerEncoder(self.encoder_layer, 1)
    self.mamba = Mamba(128)
    self.linear_2 = nn.Linear(128*49,10)



  def forward(self, x):
    x = self.linear_1(x)
    # x = self.encoder(x)
    x = self.mamba(x)
    x = x.view((-1,49*128))
    x = self.linear_2(x)
    return x



In [ ]:
vit = ViT().to(device)
x = torch.rand(size = (128,49,16)).to(device)
y = vit(x)

In [ ]:
y.shape

torch.Size([128, 10])

In [ ]:
opt = torch.optim.Adam(params = vit.parameters(), lr = 1e-3)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
def train(data, model, loss_fn, opt):

  model.train()
  size = len(data.dataset)
  for b,(x,y) in enumerate(data):
    x,y = x.to(device), y.to(device)
    y_pred = model(x)
    loss = loss_fn(y_pred, y)

    opt.zero_grad()
    loss.backward()
    opt.step()

    if b % 100 == 0:
      print(f"loss:{loss.item()} [{b*len(x)} | {size}]")




In [ ]:
def test(data, model, loss_fn):

  model.eval()
  test_loss, test_acc = 0,0
  size = len(data.dataset)
  num_batch = len(data)

  with torch.no_grad():
    for x,y in data:
      x,y = x.to(device), y.to(device)
      y_pred = model(x)
      test_loss += loss_fn(y_pred, y).item()
      test_acc += (y_pred.argmax(1)==y).type(torch.float).sum().item()


    test_loss /= num_batch
    test_acc /= size

    print(f"test loss : {test_loss} test_acc:{test_acc * 100}")

In [ ]:
for e in range(10):
  print(f"e:{e+1}=-=-=-=")
  train(train_dl, vit, loss_fn, opt)
  test(test_dl, vit, loss_fn)


e:1=-=-=-=
loss:2.3025152683258057 [0 | 60000]
loss:0.32290220260620117 [12800 | 60000]
loss:0.3095763325691223 [25600 | 60000]
loss:0.1347079575061798 [38400 | 60000]
loss:0.2218817174434662 [51200 | 60000]
test loss : 0.12128578026224918 test_acc:96.23
e:2=-=-=-=
loss:0.10321967303752899 [0 | 60000]
loss:0.3236004710197449 [12800 | 60000]
loss:0.13325367867946625 [25600 | 60000]
loss:0.06922060996294022 [38400 | 60000]
loss:0.06454239785671234 [51200 | 60000]
test loss : 0.08395919760341367 test_acc:97.21
e:3=-=-=-=
loss:0.08210990577936172 [0 | 60000]
loss:0.04477821663022041 [12800 | 60000]
loss:0.03670801222324371 [25600 | 60000]
loss:0.049624744802713394 [38400 | 60000]
loss:0.07914181798696518 [51200 | 60000]
test loss : 0.0711551619298709 test_acc:97.63
e:4=-=-=-=
loss:0.035897232592105865 [0 | 60000]
loss:0.05256902426481247 [12800 | 60000]
loss:0.06418772041797638 [25600 | 60000]
loss:0.07647466659545898 [38400 | 60000]
loss:0.0694720447063446 [51200 | 60000]
test loss : 0.06